## Alejandro Paz - Enrique Queipo de Llano
## Parte IV

In [1]:
from search import *

In [111]:
# APARTADO a)
class Muelles(Problem):
    """Problema de los muelles:
    Representaremos los estados como tuplas de 6 pilas, que a su vez serán representadas como tuplas de 1's y 0's,
    1 si el contenedor es objetivo y 0 si no lo es, las pilas 0,1,2 son las del muelle 1 y las 3,4,5 las del muelle 2
    Ademas las acciones serán representadas como tuplas (i,j) donde i es la pila de donde se coje el contenedor y j donde se 
    deja. goal_test se podría conseguir en coste logarítmico aplicando búsqueda binaria y cambiando la representación por una en la 
    que tuviera los identificadores de los contenedores junto a un valor que indique si es objetivo o no, pero consideramos 
    que no merece la pena complicar tanto la respresentación"""

    def __init__(self,initial):
        self.initial = initial
        self.numObj = 0
        self.alt = 3
        #Calcular numero de contenedores objetivo
        for i in range (len(initial)):
            for j in range(len(initial[i])):
                if (initial[i][j] == 1):
                    self.numObj +=1
        
        
    def actions(self,estado):
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs=list() 
        # Para cada pila, mover de una pila a otra distinta siempre que no se supere la altura
        for i in range(len(estado)):
            for j in range(len(estado)):
                if (i != j):
                    if (len(estado[j]) + 1 <= self.alt):
                        accs.append((i,j))
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self,estado,accion):
        # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        desde = accion[0]
        hasta = accion[1]
        estadoNuevo = []
        for i in range(len(estado)):
            # Copiar pila antigua
            estadoNuevo.append(list(estado[i])) 
            if (i == desde): # Si muevo, hago pop
                estadoNuevo[i].pop(0)
            elif (i == hasta): # Si recibe, hago insert
                estadoNuevo[i].insert(estado[desde][0],0)
        for i in range(len(estadoNuevo)):
            estadoNuevo[i] = tuple(estadoNuevo[i])
        return tuple(estadoNuevo)


    def goal_test(self,estado):
        encontrados = 0
        # Comprobar que los contenedores objetivo estan disponibles
        for i in range(3):
            for j in range(1,len(estado[i])):
                if (estado[i-1] == 0 and estado[i] == 1):
                    return False
        #Comprobar que estan todos en el muelle 1 (no me faltan)
        for i in range(3):
            for j in range(len(estado[i])):
                encontrados += estado[i][j]
                    
        return encontrados == self.numObj
    
    
    def coste_de_aplicar_accion(self, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador. El coste sera
        1 si se mueven contendores dentro de un mismo muelle y 3 si son en muelles distintos (coste == numero de maquinas que hay que accionar)"""
        i = accion[0]
        j = accion[1]
        if ((i <= 2 and j >= 3) or (i >= 3 and j <= 2)):
            return 3
        else:
            return 1
        
    

In [112]:
prueba = Muelles(((1,0),(0,0),(0,),(0,1),(0,1,0),(0,)))

In [113]:
prueba.numObj

3

In [114]:
prueba.actions(prueba.initial)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 5),
 (1, 0),
 (1, 2),
 (1, 3),
 (1, 5),
 (2, 0),
 (2, 1),
 (2, 3),
 (2, 5),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 5),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 5),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3)]

In [115]:
prueba.result(prueba.initial,(2,0))

((0, 1, 0), (0, 0), (), (0, 1), (0, 1, 0), (0,))

In [104]:
prueba.goal_test(prueba.initial)

False

In [105]:
prueba.goal_test(((1,1,1),(0,),(0,),(0,),(0,),(0,)))

True

In [108]:
prueba.coste_de_aplicar_accion((2,5))

3

In [109]:
prueba.coste_de_aplicar_accion((2,1))

1

Apartado b)
Salta a la vista rápidamente la existencia de ciclos al hacer distintas acciones, ya que se podrían repetir las acciones (0,1) (1,0) constantemente (p. ej), por este motivo se descartan de primeras los algoritmos tree_search que no controlan ciclos.
Además no parece muy razonable tampoco utilizar búsquedas ciegas, por lo que implementaremos una heurística y emplearemos el algoritmo A*

In [122]:
# Heurística contar las cajas objetivo que tengo en el muelle 2
def cajasEn2(node):
    encontrados = 0
    for i in range(3,6):
        for j in range(len(node[i])):
            encontrados += node[i][j]

    return encontrados

# Se relaja el problema pensando que se pueden coger las cajas de cualquier posicion de la pila, no solo la cima. Así el coste
# de mover cajas objetivo del muelle 2 al 1 será 3*cajasEn2 y el de hacer que las del 1 estén disponibles será 1.
def linear(node):
    suma = 3*cajasEn2(node)
    for i in range(3):
        primerCero = False
        for j in range(len(node[i])):
            if (node[i][j] == 0):
                primerCero = True
            if (primerCero):
                suma += node[i][j]
    return suma  

In [118]:
cajasEn2(((0,),(0,),(0,),(0,),(0,),(1,1,1)))

3

In [124]:
linear(((0,0,1),(0,),(1,0,0),(0,),(0,),(0,1,0)))

4